# Create Agent

### Getting Started

This sample demonstrates how to evaluate Azure AI Agent
Before running the sample:
```bash
pip install azure-ai-projects azure-identity azure-ai-evaluation
```
Set these environment variables with your own values:
1) **PROJECT_CONNECTION_STRING** - The project connection string, as found in the overview page of your Azure AI Foundry project.
2) **MODEL_DEPLOYMENT_NAME** - The deployment name of the AI model, as found under the "Name" column in the "Models + endpoints" tab in your Azure AI Foundry project.
3) **AZURE_OPENAI_ENDPOINT** - Azure Open AI Endpoint to be used for evaluation.
4) **AZURE_OPENAI_API_KEY** - Azure Open AI Key to be used for evaluation.
5) **AZURE_OPENAI_API_VERSION** - Azure Open AI Api version to be used for evaluation.


### Initializing Project Client

In [1]:

import pandas as pd
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import FunctionTool, ToolSet
from user_functions import user_functions

import os
from dotenv import load_dotenv

load_dotenv()

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

AGENT_NAME = "Seattle Tourist Assistant for bug bash"

# Adding Tools to be used by Agent 
functions = FunctionTool(user_functions)

toolset = ToolSet()
toolset.add(functions)

### Create Agent

In [2]:
agent = project_client.agents.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name=AGENT_NAME,
    instructions="You are a helpful assistant",
    toolset=toolset,
)

print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_6c9zDqpAI7hgbQQVcNrpDzM6


### Create Thread

In [3]:
thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_JLr67cGOzy6RjK4dyDxJTCzb


## Conversation with Agent
Use below cells to have conversation with the agent
- `Create Message[1]`
- `Execute[2]`

### Create Message[1]

In [4]:
# Create message to thread

MESSAGE = "Can you send me an email with weather information for Seattle?"

message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=MESSAGE,
)
print(f"Created message, ID: {message.id}")

Created message, ID: msg_H5Qvl37d7dzDieHruwh5m8sF


### Execute[2]

In [6]:
run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)

print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

print(f"Run ID: {run.id}")

Sending email to you@example.com...
Subject: Weather Information for Seattle
Body:
Weather data not available for this location.
Run finished with status: RunStatus.COMPLETED
Run ID: run_36pIVTiDXMkE8vxs2iZDk4mG


### List Messages

In [7]:
for message in project_client.agents.list_messages(thread.id, order="asc").data:
    print(f"Role: {message.role}")
    print(f"Content: {message.content[0].text.value}")
    print("-" * 40)

Role: MessageRole.USER
Content: Can you send me an email with weather information for Seattle?
----------------------------------------
Role: MessageRole.AGENT
Content: I have sent you an email with the weather information for Seattle. However, the data indicates that weather information is not available for this location. If you need anything else, just let me know!
----------------------------------------


# Evaluate

### Get data from agent

In [8]:
import json
from azure.ai.evaluation import AIAgentConverter

# Initialize the converter that will be backed by the project.
converter = AIAgentConverter(project_client)

thread_id = thread.id
run_id = run.id

converted_data = converter.convert(thread_id, run_id)
print(json.dumps(converted_data, indent=4))

{
    "query": [
        {
            "role": "system",
            "content": "You are a helpful assistant"
        },
        {
            "createdAt": "2025-03-24T18:38:31Z",
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Can you send me an email with weather information for Seattle?"
                }
            ]
        }
    ],
    "response": [
        {
            "createdAt": "2025-03-24T18:38:44Z",
            "run_id": "run_36pIVTiDXMkE8vxs2iZDk4mG",
            "role": "assistant",
            "content": [
                {
                    "type": "tool_call",
                    "tool_call_id": "call_IonwQ0BQpKQWxcVwhLywiK3J",
                    "name": "fetch_weather",
                    "arguments": {
                        "location": "Seattle"
                    }
                }
            ]
        },
        {
            "createdAt": "2025-03-24T18:38:45Z",
    

In [1]:
import pandas as pd
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import FunctionTool, ToolSet
from user_functions import user_functions
import json
import os
from dotenv import load_dotenv
from azure.ai.evaluation import AIAgentConverter

load_dotenv()

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

converter = AIAgentConverter(project_client)
# code_interpreter
thread_id = 'thread_HVSuvsEWKb1rvLVZ9w8wfPWA'
run_id = 'run_2XtFg5KIeJoWi3GF1OWfkFoU'

# bing_grounding
thread_id = "thread_LQneQpxLNEB5vmNdMoniXgBB"
run_id = "run_Kt5VWV5HEmmNesZEso5PeVhc"

converted_data = converter.convert(thread_id, run_id)
print(json.dumps(converted_data, indent=4))

{
    "query": [
        {
            "role": "system",
            "content": "Be helpful."
        },
        {
            "createdAt": "2025-03-24T19:14:13Z",
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "What are side effects of the typhoid vaccine?"
                }
            ]
        },
        {
            "createdAt": "2025-03-24T19:14:14Z",
            "run_id": "run_385Q4kLjYWZk7wrje28WlWv4",
            "role": "assistant",
            "content": [
                {
                    "type": "text",
                    "text": "The typhoid vaccine, which is available in injectable and oral forms, can have a range of side effects. These side effects can vary depending on the form of the vaccine used. Here are some common and less common side effects:\n\n1. **Injectable Typhoid Vaccine (Vi polysaccharide vaccine):**\n   - **Common Side Effects:**\n     - Pain or swelling at the i

In [4]:


evaluation_data = []
run_ids = converter._list_run_ids_chronological(thread_id=thread.id)

for run_id in run_ids:
    run_data = converter.convert(thread_id, run_id)
    evaluation_data.append(run_data)

# write to file
file_path = os.path.join(os.getcwd(), "evaluation_data.jsonl")
pd.DataFrame(evaluation_data).to_json(
    file_path, orient="records", lines=True
)

NameError: name 'thread' is not defined

### Setting up evaluator

In [ ]:
from azure.ai.evaluation import ToolCallAccuracyEvaluator , AzureOpenAIModelConfiguration, IntentResolutionEvaluator, TaskAdherenceEvaluator
from pprint import pprint

model_config = AzureOpenAIModelConfiguration(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["MODEL_DEPLOYMENT_NAME"],
)

tool_call_accuracy = ToolCallAccuracyEvaluator(model_config=model_config)
intent_resolution = IntentResolutionEvaluator(model_config=model_config)
task_adherence = TaskAdherenceEvaluator(model_config=model_config)

### Run Evaluator

In [ ]:
from azure.ai.evaluation import evaluate

response = evaluate(
    data=file_path,
    evaluators={
        "tool_call_accuracy": tool_call_accuracy,
        "intent_resolution": intent_resolution,
        "task_adherence": task_adherence,
    },
    azure_ai_project={
        "subscription_id": os.environ["SUBSCRIPTION_ID"],
        "project_name": os.environ["PROJECT_NAME"]",
        "resource_group_name": os.envirn["RESOURCE_GROUP_NAME"],
    }
)
pprint(f'AI Foundary URL: {response.get("studio_url")}')

In [ ]:
pprint(response)